In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install tensorflow==2.5.0

In [ ]:
import os

In [ ]:
import tensorflow as tf
tf.version.VERSION

'2.3.0'

In [ ]:
os.chdir("/content/drive/MyDrive/cr/crfill/networks")

In [ ]:
from convnet_tf import InpaintGenerator_tf,InpaintGenerator_tf_new ,contextual_module

## Load weight for contextual 

In [ ]:
contextual = contextual_module(shape = (64,64,96),new_shape = (256,256,1))

In [ ]:
# import torch
# model = torch.load("/content/drive/MyDrive/generative_inpaintingv2/contextual_loss/crfill/checkpoint/InpaintGenerator_gen_full.pth",map_location=torch.device('cpu'))
import torch
model = torch.load("/content/drive/MyDrive/generative_inpaintingv2/contextual_loss_fix_mask_wise_and_pixel_shuffle/checkpoint/checkpoint_change_dis/backup_after_change_batch/InpaintGenerator_gen_full.pth",map_location=torch.device('cpu'))

In [ ]:
count1 =0 
for k,v in model["generator_full"].items():
   
    print("{:20s} {}".format(k, v.shape))
    count1 += 1

baseg.conv1.weight   torch.Size([24, 5, 5, 5])
baseg.conv1.bias     torch.Size([24])
baseg.conv1.mask_conv2d.weight torch.Size([1, 5, 5, 5])
baseg.conv1.mask_conv2d.bias torch.Size([1])
baseg.conv2_downsample.weight torch.Size([48, 24, 3, 3])
baseg.conv2_downsample.bias torch.Size([48])
baseg.conv2_downsample.mask_conv2d.weight torch.Size([1, 24, 3, 3])
baseg.conv2_downsample.mask_conv2d.bias torch.Size([1])
baseg.conv3.weight   torch.Size([48, 48, 3, 3])
baseg.conv3.bias     torch.Size([48])
baseg.conv3.mask_conv2d.weight torch.Size([1, 48, 3, 3])
baseg.conv3.mask_conv2d.bias torch.Size([1])
baseg.conv4_downsample.weight torch.Size([96, 48, 3, 3])
baseg.conv4_downsample.bias torch.Size([96])
baseg.conv4_downsample.mask_conv2d.weight torch.Size([1, 48, 3, 3])
baseg.conv4_downsample.mask_conv2d.bias torch.Size([1])
baseg.conv5.weight   torch.Size([96, 96, 3, 3])
baseg.conv5.bias     torch.Size([96])
baseg.conv5.mask_conv2d.weight torch.Size([1, 96, 3, 3])
baseg.conv5.mask_conv2d.bias to

In [ ]:
count1

210

In [ ]:
count1 =0  
for layer in contextual.layers:
  if layer.get_weights():
    bias = model["generator_full"][str(layer.name)+".bias"].detach().numpy()
    weight = model["generator_full"][str(layer.name)+".weight"].detach().numpy()
#             bias1 = model["generator_base"][str(layer.layer.layer.name)+".bias"].detach().numpy()
    # print(str(layer.name))
    
    weight = weight.transpose((2, 3, 1, 0))
    layer.set_weights([weight,bias])
    count1 += 1 

In [ ]:
count1

4

In [ ]:
contextual.input[0].set_shape([1, 64, 64, 96])
contextual.input[1].set_shape([1, 256, 256, 1])
contextual.input[2].set_shape([1,1])
contextual.input[3].set_shape([1,1])

In [ ]:
contextual.save("contextual_update1.h5")

In [ ]:
import tensorflow as tf
contextual = tf.keras.models.load_model('contextual_update1.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(contextual)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops = True
converter.experimental_new_converter = True

In [ ]:
!pip install tf-nightly

In [ ]:
tflite_convert = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpjcdfdlsz/assets


INFO:tensorflow:Assets written to: /tmp/tmpjcdfdlsz/assets


In [ ]:
with open('./contextual_update_1.tflite', 'wb') as f:
  f.write(tflite_convert)

In [ ]:
tflite = tf.lite.Interpreter(model_path="./contextual_update_1.tflite")

In [ ]:
input_details = tflite.get_input_details()
output_details = tflite.get_output_details()

In [ ]:
tflite.resize_tensor_input(
    input_details[0]['index'], (1, 128, 128, 96))
tflite.resize_tensor_input(
    input_details[1]['index'], (1, 512, 512, 3))

In [ ]:
tflite.allocate_tensors()

## End load weight

## Our inpaint model

In [ ]:
model_tf_new = InpaintGenerator_tf_new(shape =(256,256,3),new_shape =(256,256,1) )

In [ ]:
for layer in model_tf_new.layers:
    if layer.get_weights():
        print(layer.name)

In [ ]:
import torch
# model =torch.load("/content/drive/MyDrive/generative_inpaintingv2/contextual_loss_mask_wise_fix_pixel_shuffle/checkpoint/checkpoint_change_dis/InpaintGenerator_gen_base.pth",map_location=torch.device('cpu'))
# model =torch.load("/content/drive/MyDrive/generative_inpaintingv2/contextual_loss_fix_mask_wise_and_pixel_shuffle/checkpoint/InpaintGenerator_gen_base.pth",map_location=torch.device('cpu'))
# model =torch.load("/content/drive/MyDrive/generative_inpaintingv2/contextual_loss_fix_mask_wise_and_pixel_shuffle/checkpoint/checkpoint_change_dis/InpaintGenerator_gen_full.pth",map_location=torch.device('cpu'))
model =torch.load("/content/drive/MyDrive/generative_inpaintingv2/contextual_loss_fix_mask_wise_and_pixel_shuffle/checkpoint/checkpoint_change_dis/backup_after_change_batch/InpaintGenerator_gen_full.pth",map_location=torch.device('cpu'))


In [ ]:
count1 =0 
for k,v in model["generator_full"].items():
   
    print("{:20s} {}".format(k, v.shape))
    count1 += 1

baseg.conv1.weight   torch.Size([24, 5, 5, 5])
baseg.conv1.bias     torch.Size([24])
baseg.conv1.mask_conv2d.weight torch.Size([1, 5, 5, 5])
baseg.conv1.mask_conv2d.bias torch.Size([1])
baseg.conv2_downsample.weight torch.Size([48, 24, 3, 3])
baseg.conv2_downsample.bias torch.Size([48])
baseg.conv2_downsample.mask_conv2d.weight torch.Size([1, 24, 3, 3])
baseg.conv2_downsample.mask_conv2d.bias torch.Size([1])
baseg.conv3.weight   torch.Size([48, 48, 3, 3])
baseg.conv3.bias     torch.Size([48])
baseg.conv3.mask_conv2d.weight torch.Size([1, 48, 3, 3])
baseg.conv3.mask_conv2d.bias torch.Size([1])
baseg.conv4_downsample.weight torch.Size([96, 48, 3, 3])
baseg.conv4_downsample.bias torch.Size([96])
baseg.conv4_downsample.mask_conv2d.weight torch.Size([1, 48, 3, 3])
baseg.conv4_downsample.mask_conv2d.bias torch.Size([1])
baseg.conv5.weight   torch.Size([96, 96, 3, 3])
baseg.conv5.bias     torch.Size([96])
baseg.conv5.mask_conv2d.weight torch.Size([1, 96, 3, 3])
baseg.conv5.mask_conv2d.bias to

In [ ]:
count1

210

In [ ]:
from utils_tf import gen_conv1,gen_deconv1,gen_conv2,gen_deconv2

In [ ]:
count = 0

for layer in model_tf_new.layers:
    if layer.get_weights():
        weight = model["generator_full"]["baseg."+str(layer.name)+".weight"].numpy()
        bias = model["generator_full"]["baseg."+str(layer.name)+".bias"].numpy()
        weight = weight.transpose((2, 3, 1, 0))

        layer.set_weights([weight,bias])
        count += 1

In [ ]:
count

82

In [ ]:
model_tf_new.input[0].set_shape([1, 256, 256, 3])
model_tf_new.input[1].set_shape([1, 256, 256, 1])
model_tf_new.input[2].set_shape([1, 256, 256, 1])

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model_tf_new)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops = True
converter.experimental_new_converter = True
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpr7jljd2h/assets


INFO:tensorflow:Assets written to: /tmp/tmpr7jljd2h/assets


In [ ]:
with open('./cr_lwgcoc_latest1.tflite', 'wb') as f:
  f.write(tflite_model)

## End inpaint new

## Pretrained high resolution

In [ ]:
model_tf= InpaintGenerator_tf(shape =(512,512,3),new_shape =(512,512,1) )

In [ ]:
model_tf.summary()

Model: "inpaintgenerator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_RealDiv (TensorFlow [(None, 512, 512, 3) 0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
tf_op_layer_Sub (TensorFlowOpLa [(None, 512, 512, 3) 0           tf_op_layer_RealDiv[0][0]        
___________________________________________________________________________________

In [ ]:
import torch
model = torch.load("/content/drive/MyDrive/cr/crfill/files/model_near512.pth",map_location=torch.device('cpu'))
# model = torch.load("/content/drive/MyDrive/cr/crfill/files/model_256.pth",map_location=torch.device('cpu'))


In [ ]:

count1 =0 
for k,v in model.items():
   
    print("{:20s} {}".format(k, v.shape))
    count1 += 1

conv1.weight         torch.Size([48, 5, 5, 5])
conv1.bias           torch.Size([48])
conv2_downsample.weight torch.Size([96, 24, 3, 3])
conv2_downsample.bias torch.Size([96])
conv3.weight         torch.Size([96, 48, 3, 3])
conv3.bias           torch.Size([96])
conv4_downsample.weight torch.Size([192, 48, 3, 3])
conv4_downsample.bias torch.Size([192])
conv5.weight         torch.Size([192, 96, 3, 3])
conv5.bias           torch.Size([192])
conv6.weight         torch.Size([192, 96, 3, 3])
conv6.bias           torch.Size([192])
conv7_atrous.weight  torch.Size([192, 96, 3, 3])
conv7_atrous.bias    torch.Size([192])
conv8_atrous.weight  torch.Size([192, 96, 3, 3])
conv8_atrous.bias    torch.Size([192])
conv9_atrous.weight  torch.Size([192, 96, 3, 3])
conv9_atrous.bias    torch.Size([192])
conv10_atrous.weight torch.Size([192, 96, 3, 3])
conv10_atrous.bias   torch.Size([192])
conv11.weight        torch.Size([192, 96, 3, 3])
conv11.bias          torch.Size([192])
conv12.weight        torch.Size

In [ ]:
count1

84

In [ ]:
# count
count = 0
# a = 0
for layer in model_tf.layers:

    if layer.get_weights():
        weight = model[str(layer.name)+".weight"].numpy()
        bias = model[str(layer.name)+".bias"].numpy()
        weight = weight.transpose((2, 3, 1, 0))
       
        layer.set_weights([weight,bias])
        count += 1

In [ ]:
count

42

## Convert tf-lite

In [ ]:
model_tf.input[0].set_shape([1, 512, 512, 3])
model_tf.input[1].set_shape([1, 512, 512, 1])
model_tf.input[2].set_shape([1, 512, 512, 1])

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops = True
converter.experimental_new_converter = True

In [ ]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmprewzxp5g/assets


INFO:tensorflow:Assets written to: /tmp/tmprewzxp5g/assets


In [ ]:
with open('./cr_origin_high_res.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
tflite = tf.lite.Interpreter(model_path="./cr_origin_high_res.tflite")

In [ ]:
input_details = tflite.get_input_details()
output_details = tflite.get_output_details()

In [ ]:
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'input_1',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 512, 512,   3], dtype=int32),
  'shape_signature': array([  1, 512, 512,   3], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 1,
  'name': 'input_2',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 512, 512,   1], dtype=int32),
  'shape_signature': array([  1, 512, 512,   1], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 2,
  'name': 'input_3',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 512,

In [ ]:
tflite.resize_tensor_input(
    input_details[0]['index'], (1, 512, 512, 3))
tflite.resize_tensor_input(
    input_details[1]['index'], (1, 512, 512, 1))
tflite.resize_tensor_input(
    input_details[2]['index'], (1, 512, 512, 1))

In [ ]:
tflite.get_output_details()

[{'dtype': numpy.float32,
  'index': 421,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 512, 512,   3], dtype=int32),
  'shape_signature': array([ -1, 512, 512,   3], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 363,
  'name': 'Identity_1',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 128, 128,  96], dtype=int32),
  'shape_signature': array([ -1, 128, 128,  96], dtype=int32),
  'sparsity_parameters': {}}]